In [9]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import importlib
import scripts.theodolite_utils as theodo_u
import scripts.theodolite_function as theodo_f
import scripts.prediction_utils as prediction_u
import scripts.resection_functions as theodo_r
import time
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt

np.set_printoptions(6, suppress=True)

from pylgmath import Transformation, se3op
from pysteam.trajectory import Time
from pysteam.trajectory.const_vel import Interface as TrajectoryInterface
from pysteam.problem import OptimizationProblem, StaticNoiseModel, L2LossFunc, WeightedLeastSquareCostTerm
from pysteam.solver import GaussNewtonSolver, DoglegGaussNewtonSolver
from pysteam.evaluable import vspace as vspaceev, se3 as se3ev
from pysteam.evaluable.se3 import SE3StateVar
from pysteam.evaluable.vspace import VSpaceStateVar

ModuleNotFoundError: No module named 'scripts'

In [ ]:
theodo_u = importlib.reload(theodo_u)

# Path of the RTS data
path = "../data/20220224/"

# Read Extrinsic results for RTS
Tf = theodo_u.read_saved_tf(path+"list_tf/TF_list_static_cp.csv")
#Tf = theodo_u.read_saved_tf(path+"list_tf/TF_list_linear_dynamic_4dof.csv")
Tf_1 = Tf[0]
Tf_12 = Tf[1]
Tf_13 = Tf[2]

# Reading sensor data
Sensor = "GNSS"   # GNSS or Robosense_32
# path_sensor_file = path+"ICP/icp_odom.csv"

# Sensor = "GNSS"
path_sensor_file = path+"gps_data/"+"gpsmiddle.txt"
path_sensor_file_synch_time = path+"gps_data/delay_synchronization_GNSS_3.txt"  # If applied to GNSS
Gps_reference_chosen = 3    # 1: front, 2: back, 3: middle   # Only for GNSS

if Sensor == "GNSS":
    GNSS_raw_data = theodo_u.read_prediction_data_Linear_csv_file(path_sensor_file)
    time_delay = float(theodo_u.read_time_delay(path_sensor_file_synch_time))
    sensor_data = []
    for i in GNSS_raw_data:
        raw_data = np.array([i[0]+time_delay, i[1], i[2], i[3], 0, 0, 0, 1])
        sensor_data.append(raw_data)
    sensor_data = np.array(sensor_data)
if Sensor=="Robosense_32":
    sensor_data = theodo_u.read_icp_odom_file(path_sensor_file)
    sensor_data = np.array(sensor_data)

In [ ]:
# Path of the rosbags to process the RTS data
file = [
        "/home/maxime/data/ICRA_2023/Vaidis2022_dataset/20220224/20220224_inter_prism.bag"
       ]

# Parameters to select:
# 1. Apply filtering or not (Module 1)
# 2-3-4. Parameters tau_r, tau_a, tau_e (Module 1)
# 5. Parameter tau_s (Module 2)

parameters = [
    [1,2,2,2,4],
]

# Path of output
output = [
        path
]

In [ ]:
theodo_u = importlib.reload(theodo_u)
theodo_f = importlib.reload(theodo_f)
prediction_u = importlib.reload(prediction_u)

for param in parameters:
    print(param)

    if(param[0]==0):
        filtering = False
    if(param[0]==1):
        filtering = True
    thresold_d = param[1]                # tau_r [m/s]
    thresold_a = param[2]                # tau_a [deg/s]
    thresold_e = param[3]                # tau_e [deg/s]
    limit_time_interval = param[4]       # tau_s [s]
    size_interval = 2

    Mode = "L"       # Interpolation choice: 1. L -> Linear interpolation, 2. SGP -> Gaussian Process with Stheno library
    limit_search = limit_time_interval
    save = False

    save_index_1 = []
    save_index_2 = []
    save_index_3 = []

    for fname, opath in zip(file,output):
        if(not filtering):
            path_out = opath + "raw_prediction/"
        else:
            path_out = opath + "filtered_prediction/"

        if(filtering):
            t1, t2, t3, tp1, tp2, tp3, d1, d2, d3, a1, a2, a3, e1, e2, e3 = theodo_u.read_rosbag_theodolite_without_tf_raw_data_pre_filtered(fname)
            index_1_f = theodo_u.thresold_raw_data(t1, d1, a1, e1, thresold_d, thresold_a*3.1415926/180, thresold_e*3.1415926/180, limit_time_interval)
            index_2_f = theodo_u.thresold_raw_data(t2, d2, a2, e2, thresold_d, thresold_a*3.1415926/180, thresold_e*3.1415926/180, limit_time_interval)
            index_3_f = theodo_u.thresold_raw_data(t3, d3, a3, e3, thresold_d, thresold_a*3.1415926/180, thresold_e*3.1415926/180, limit_time_interval)
            t1 = t1[index_1_f]
            t2 = t2[index_2_f]
            t3 = t3[index_3_f]
            tp1 = tp1[index_1_f].T
            tp2 = tp2[index_2_f].T
            tp3 = tp3[index_3_f].T
            print(len(t1),len(t2),len(t3))
        else:
            t1, t2, t3, tp1, tp2, tp3, d1, d2, d3, a1, a2, a3, e1, e2, e3 = theodo_u.read_rosbag_theodolite_without_tf_raw_data(fname)
            print(len(t1),len(t2),len(t3))

        start_time = time.time()

        # Put trajectories in same frame
        tp1 = Tf_1@tp1
        tp2 = Tf_12@tp2
        tp3 = Tf_13@tp3

        list_interval, list_time = theodo_f.split_time_interval_all_data(t1, t2, t3, limit_time_interval)
        list_trajectories_split = theodo_f.merge_interval(list_interval, list_time, t1, t2, t3, limit_search)

        Prediction_1 = []
        Prediction_2 = []
        Prediction_3 = []
        T_prediction = []
        Index_sensor = []

        for i in tqdm(list_trajectories_split):

            index_1 = np.array([i[0,0],i[1,0]])
            index_2 = np.array([i[0,1],i[1,1]])
            index_3 = np.array([i[0,2],i[1,2]])

            save_index_1.append(index_1)
            save_index_2.append(index_2)
            save_index_3.append(index_3)

            begin = np.max([t1[index_1[0]], t2[index_2[0]], t3[index_3[0]]])
            end = np.min([t1[index_1[1]], t2[index_2[1]], t3[index_3[1]]])

            if abs(end-begin)>size_interval and begin<end:

                Number = 0
                T_prediction_sensor = []
                for value_sensor_data in sensor_data:
                    if end >= value_sensor_data[0] >= begin:
                        T_prediction_sensor.append(value_sensor_data[0])
                        Index_sensor.append(Number)
                    Number = Number + 1

                #rate = 10  #Hz
                #T_prediction_init = torch.from_numpy(np.arange(begin, end, 1/rate))
                #print(T_prediction_sensor)
                T_prediction_init = torch.from_numpy(np.array(T_prediction_sensor))

                # Linear interpolation
                if Mode == "L" or Mode == "All":
                    T1, X1, Y1, Z1, T2, X2, Y2, Z2, T3, X3, Y3, Z3 = prediction_u.data_training_L(t1, t2, t3, tp1, tp2, tp3, index_1, index_2, index_3)
                    mx1, my1, mz1, mx2, my2, mz2, mx3, my3, mz3 = prediction_u.linear_interpolation(T1, X1, Y1, Z1, T2, X2, Y2, Z2,T3, X3, Y3, Z3)

                    for i in T_prediction_init.numpy():
                        T_prediction.append(i)
                        P1_L, P2_L, P3_L = prediction_u.linear_prediction(i, 0, mx1, my1, mz1, mx2, my2, mz2, mx3, my3, mz3)
                        Prediction_1.append(P1_L)
                        Prediction_2.append(P2_L)
                        Prediction_3.append(P3_L)

        stop_time = time.time()
        print(stop_time - start_time)

        print("Interpolation finished !")

        if save:

            if(Mode == "L" or Mode == "All"):
                if(filtering):
                    trajectoire = "f-"+str(thresold_d)+"-"+str(thresold_a)+"-"+str(thresold_e)+"-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+"-L"
                else:
                    trajectoire = "nf-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+"-L"

                if save:
                    theodo_u.Convert_raw_data_point_to_csv(T_prediction, Prediction_1, path_out+trajectoire+ "_1.csv")
                    theodo_u.Convert_raw_data_point_to_csv(T_prediction, Prediction_2, path_out+trajectoire+ "_2.csv")
                    theodo_u.Convert_raw_data_point_to_csv(T_prediction, Prediction_3, path_out+trajectoire+ "_3.csv")

            print("Saved !")

# Trajectories predicted
Time_p = np.array(T_prediction)
P1_arr = np.array(Prediction_1)
P2_arr = np.array(Prediction_2)
P3_arr = np.array(Prediction_3)
Index_sensor = np.array(Index_sensor)

In [ ]:
theodo_r = importlib.reload(theodo_r)
file_sensors = theodo_u.if_file_exist(path + "sensors_extrinsic_calibration/calibration_results.csv",'')
extrinsic_calibration_results = theodo_u.read_extrinsic_calibration_results_file(file_sensors)
error=[]
error = theodo_r.inter_prism_distance_error_mean(P1_arr, P2_arr, P3_arr, extrinsic_calibration_results[0:3])

In [ ]:
# Outliers for results
P1_sort = []
P2_sort = []
P3_sort = []
Time_sort = []
Index_sensor_sort = []
threshold_inter_prims_error = 10    # In mm, need to check how to choose it
for i,j,k,l,m,n in zip(error,P1_arr,P2_arr,P3_arr,T_prediction,Index_sensor):
    if i<threshold_inter_prims_error:
        P1_sort.append(j)
        P2_sort.append(k)
        P3_sort.append(l)
        Time_sort.append(m)
        Index_sensor_sort.append(n)
print("Number points input/output: ", len(P1_arr), len(P1_sort))

In [ ]:
# Load sensor positions
file_sensor_positions = theodo_u.if_file_exist(path + "sensors_extrinsic_calibration/sensor_positions.csv",'')
sensor_positions_list = theodo_u.read_extrinsic_calibration_results_file(file_sensor_positions)

Sensors = []
P1_position_RTS = np.array(sensor_positions_list[0])
P2_position_RTS = np.array(sensor_positions_list[1])
P3_position_RTS = np.array(sensor_positions_list[2])
if Sensor=="GNSS":
    Sensors.append(np.array(sensor_positions_list[3]))
    Sensors.append(np.array(sensor_positions_list[4]))
    Sensors.append(np.array(sensor_positions_list[5]))

    P1_position_GNSS = P1_position_RTS - Sensors[Gps_reference_chosen-1]
    P1_position_GNSS[3] = 1
    P2_position_GNSS = P2_position_RTS - Sensors[Gps_reference_chosen-1]
    P2_position_GNSS[3] = 1
    P3_position_GNSS = P3_position_RTS - Sensors[Gps_reference_chosen-1]
    P3_position_GNSS[3] = 1

    P_sensor = np.array([P1_position_GNSS,
                P2_position_GNSS,
                P3_position_GNSS]).T

if Sensor=="Robosense_32":
    Sensors.append(np.array(sensor_positions_list[6]))
    Sensors.append(np.array(sensor_positions_list[7]))
    Sensors.append(np.array(sensor_positions_list[8]))
    Sensors.append(np.array(sensor_positions_list[9]))
    ux = Sensors[1] - Sensors[0]
    uy = Sensors[2] - Sensors[0]
    uz = Sensors[3] - Sensors[0]

    T_lidar = np.array([[ux[0],uy[0],uz[0],Sensors[0][0]],
                        [ux[1],uy[1],uz[1],Sensors[0][1]],
                        [ux[2],uy[2],uz[2],Sensors[0][2]],
                        [0,0,0,1]])
    T_lidar_inv = np.linalg.inv(T_lidar)
    P1_position_lidar = T_lidar_inv@P1_position_RTS
    P2_position_lidar = T_lidar_inv@P2_position_RTS
    P3_position_lidar = T_lidar_inv@P3_position_RTS

    P_sensor = np.array([P1_position_lidar,
                P2_position_lidar,
                P3_position_lidar]).T


In [ ]:
# Doing a minimization between these not moving points, and the 3D prism coordinates
# Pose_GNSS is a list of each rigid transform founded
list_sensor_time = []
Pose_sensor = []
Prism_corrected = []
number = len(P1_sort)
for i in range(0,number):
    Q = np.array([P1_sort[i], P2_sort[i], P3_sort[i]]).T
    Q =np.concatenate((Q, np.array([[1,1,1]])), axis=0)
    T = theodo_u.point_to_point_minimization(P_sensor, Q)
    Pose_sensor.append(T)
    prism_correct = T@P_sensor
    Prism_corrected.append(prism_correct)
    list_sensor_time.append(Time_sort[i])
Pose_sensor_arr = np.array(Pose_sensor)
Prism_corrected_arr = np.array(Prism_corrected)

In [ ]:
theodo_u = importlib.reload(theodo_u)
theodo_f = importlib.reload(theodo_f)
prediction_u = importlib.reload(prediction_u)

for param in parameters:
    print(param)

    if(param[0]==0):
        filtering = False
    if(param[0]==1):
        filtering = True
    thresold_d = param[1]                # tau_r [m/s]
    thresold_a = param[2]                # tau_a [deg/s]
    thresold_e = param[3]                # tau_e [deg/s]
    limit_time_interval = param[4]       # tau_s [s]
    size_interval = 2

    Mode = "L"       # Interpolation choice: 1. L -> Linear interpolation, 2. SGP -> Gaussian Process with Stheno library
    limit_search = limit_time_interval
    save = False

    save_index_1 = []
    save_index_2 = []
    save_index_3 = []

    for fname, opath in zip(file,output):
        if(not filtering):
            path_out = opath + "raw_prediction/"
        else:
            path_out = opath + "filtered_prediction/"

        if(filtering):
            t1, t2, t3, tp1, tp2, tp3, d1, d2, d3, a1, a2, a3, e1, e2, e3 = theodo_u.read_rosbag_theodolite_without_tf_raw_data_pre_filtered(fname)
            index_1_f = theodo_u.thresold_raw_data(t1, d1, a1, e1, thresold_d, thresold_a*3.1415926/180, thresold_e*3.1415926/180, limit_time_interval)
            index_2_f = theodo_u.thresold_raw_data(t2, d2, a2, e2, thresold_d, thresold_a*3.1415926/180, thresold_e*3.1415926/180, limit_time_interval)
            index_3_f = theodo_u.thresold_raw_data(t3, d3, a3, e3, thresold_d, thresold_a*3.1415926/180, thresold_e*3.1415926/180, limit_time_interval)
            t1 = t1[index_1_f]
            t2 = t2[index_2_f]
            t3 = t3[index_3_f]
            tp1 = tp1[index_1_f].T
            tp2 = tp2[index_2_f].T
            tp3 = tp3[index_3_f].T
            print(len(t1),len(t2),len(t3))
        else:
            t1, t2, t3, tp1, tp2, tp3, d1, d2, d3, a1, a2, a3, e1, e2, e3 = theodo_u.read_rosbag_theodolite_without_tf_raw_data(fname)
            print(len(t1),len(t2),len(t3))

        T_measure = np.concatenate([t1, t2, t3])
        print(len(t1), len(t2), len(t3))
        print(len(T_measure))

        start_time = time.time()

        # Put trajectories in same frame
        tp1 = Tf_1@tp1
        tp2 = Tf_12@tp2
        tp3 = Tf_13@tp3

        list_interval, list_time = theodo_f.split_time_interval_all_data(t1, t2, t3, limit_time_interval)
        list_trajectories_split = theodo_f.merge_interval(list_interval, list_time, t1, t2, t3, limit_search)

        Prediction_1 = []
        Prediction_2 = []
        Prediction_3 = []
        T_prediction = []
        Index_sensor = []

        for i in tqdm(list_trajectories_split):

            index_1 = np.array([i[0,0],i[1,0]])
            index_2 = np.array([i[0,1],i[1,1]])
            index_3 = np.array([i[0,2],i[1,2]])

            save_index_1.append(index_1)
            save_index_2.append(index_2)
            save_index_3.append(index_3)

            begin = np.max([t1[index_1[0]], t2[index_2[0]], t3[index_3[0]]])
            end = np.min([t1[index_1[1]], t2[index_2[1]], t3[index_3[1]]])

            if abs(end-begin)>size_interval and begin<end:

                Number = 0
                T_prediction_sensor = []
                for value_sensor_data in sensor_data:
                    if end >= value_sensor_data[0] >= begin:
                        T_prediction_sensor.append(value_sensor_data[0])
                        Index_sensor.append(Number)
                    Number = Number + 1

                #rate = 10  #Hz
                #T_prediction_init = torch.from_numpy(np.arange(begin, end, 1/rate))
                #print(T_prediction_sensor)
                T_prediction_init = torch.from_numpy(np.array(T_prediction_sensor))

                # Linear interpolation
                if Mode == "L" or Mode == "All":
                    T1, X1, Y1, Z1, T2, X2, Y2, Z2, T3, X3, Y3, Z3 = prediction_u.data_training_L(t1, t2, t3, tp1, tp2, tp3, index_1, index_2, index_3)
                    mx1, my1, mz1, mx2, my2, mz2, mx3, my3, mz3 = prediction_u.linear_interpolation(T1, X1, Y1, Z1, T2, X2, Y2, Z2,T3, X3, Y3, Z3)

                    for i in T_prediction_init.numpy():
                        T_prediction.append(i)
                        P1_L, P2_L, P3_L = prediction_u.linear_prediction(i, 0, mx1, my1, mz1, mx2, my2, mz2, mx3, my3, mz3)
                        Prediction_1.append(P1_L)
                        Prediction_2.append(P2_L)
                        Prediction_3.append(P3_L)

        stop_time = time.time()
        print(stop_time - start_time)

        print("Interpolation finished !")

        if save:

            if(Mode == "L" or Mode == "All"):
                if(filtering):
                    trajectoire = "f-"+str(thresold_d)+"-"+str(thresold_a)+"-"+str(thresold_e)+"-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+"-L"
                else:
                    trajectoire = "nf-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+"-L"

                if save:
                    theodo_u.Convert_raw_data_point_to_csv(T_prediction, Prediction_1, path_out+trajectoire+ "_1.csv")
                    theodo_u.Convert_raw_data_point_to_csv(T_prediction, Prediction_2, path_out+trajectoire+ "_2.csv")
                    theodo_u.Convert_raw_data_point_to_csv(T_prediction, Prediction_3, path_out+trajectoire+ "_3.csv")

            print("Saved !")

# Trajectories predicted
Time_p = np.array(T_prediction)
P1_arr = np.array(Prediction_1)
P2_arr = np.array(Prediction_2)
P3_arr = np.array(Prediction_3)
Index_sensor = np.array(Index_sensor)

# STEAM

In [ ]:
print(len(Pose_sensor_arr))
print(len(Time_sort))

before = Time_sort[0]
index_gap = []
index = 0
index_init = 0
for i in Time_sort:
    if(abs(i-before)>4):
        index_gap.append([index_init,index-1])
        index_init = index
    before = i
    index = index + 1
index_gap.append([index_init,index-1])
# print(Time_sort[index_gap[0]-1], Time_sort[index_gap[0]])
print(index_gap[0])

In [ ]:
theodo_u = importlib.reload(theodo_u)
theodo_u.read_rosbag_imu_node("/home/maxime/data/ICRA_2023/Vaidis2022_dataset/20220224/20220224_inter_prism/", True)

In [ ]:
## Qc
qcd = np.ones(6)
## states
states = []
for i in range(index_gap[0][0], index_gap[0][1]+1):
    states.append([Time_sort[i], Transformation(T_ba=Pose_sensor_arr[i]), np.zeros((6, 1))])
state_vars = [(t, SE3StateVar(T_vi), VSpaceStateVar(w_iv_inv)) for t, T_vi, w_iv_inv in states]
state_vars[0][1].locked = True  # lock first pose
state_vars[0][2].locked = True  # lock first velocity
state_vars[-1][1].locked = True  # lock last pose
state_vars[-1][2].locked = True  # lock last velocity

## pose measurements
T_iv_noise = np.diag([1e-3, 1e-3, 1e-3, 1e-3, 1e-3, 1e-3])
T_iv_meas = []
T_iv_meas.append((1.0 * T_TOTAL, Transformation(xi_ab=np.array([[0.2, 0.05, 0.15, -0.1, -0.8, 0.6]]).T)))
## velocity measurements
w_iv_inv_noise = np.diag([1e0, 1e0, 1e0, 1e0, 1e0, 1e0])
w_iv_inv_meas = []

In [ ]:
traj = TrajectoryInterface(qcd=qcd)
for t_l, T_vi, w_iv_inv in state_vars:
    traj.add_knot(time=Time(t_l), T_k0=T_vi, w_0k_ink=w_iv_inv)
#
cost_terms = []
loss_func = L2LossFunc()
# pose measurements -> cost terms
noise_model = StaticNoiseModel(T_iv_noise)
for t_l, T_iv in T_iv_meas:
    intp_pos = traj.get_pose_interpolator(Time(secs=t_l))
    error_func = se3ev.tran2vec(se3ev.compose(intp_pos, SE3StateVar(T_iv, locked=True)))
    cost_term = WeightedLeastSquareCostTerm(error_func, noise_model, loss_func)
    cost_terms.append(cost_term)
# velocity measurements -> cost terms
noise_model = StaticNoiseModel(w_iv_inv_noise)
for t_l, w in w_iv_inv_meas:
    intp_vel = traj.get_velocity_interpolator(Time(secs=t_l))
    error_func = vspaceev.add(VSpaceStateVar(w, locked=True), vspaceev.neg(intp_vel))
    cost_term = WeightedLeastSquareCostTerm(error_func, noise_model, loss_func)
    cost_terms.append(cost_term)
#
opt_prob = OptimizationProblem()
opt_prob.add_state_var(*[v for state_var in state_vars for v in state_var[1:]])
opt_prob.add_cost_term(*traj.get_prior_cost_terms())
opt_prob.add_cost_term(*cost_terms)

solver = DoglegGaussNewtonSolver(opt_prob, verbose=False)
solver.optimize()

In [ ]:
# knot times
knot_times = [x[0] for x in state_vars]
knot_vels = np.array([traj.get_velocity_interpolator(Time(secs=t)).evaluate().flatten() for t in knot_times])
knot_poses = [traj.get_pose_interpolator(Time(secs=t)).evaluate().inverse().matrix() for t in knot_times]
knot_poses_vec = np.array([se3op.tran2vec(x).flatten() for x in knot_poses])

# measurement values
meas_pose_times = [x[0] for x in T_iv_meas]
meas_poses = [x[1].matrix() for x in T_iv_meas]
meas_poses_vec = np.array([se3op.tran2vec(x).flatten() for x in meas_poses])
meas_vel_times = [x[0] for x in w_iv_inv_meas]
meas_vels = np.array([x[1].flatten() for x in w_iv_inv_meas])